In [1]:
from collections import deque
from functools import cache

import numpy as np
from tqdm import tqdm
from scipy.spatial.distance import cdist

In [2]:
def parse_input(file):
    with open(file) as file_in:
        grid = file_in.read().splitlines()

    grid = np.array([list(row) for row in grid])
    n_rows, n_cols = grid.shape

    S_coords = tuple(np.argwhere(grid == 'S')[0].tolist())
    E_coords = tuple(np.argwhere(grid == 'E')[0].tolist())
    wall_coords = set([(x.item(), y.item()) for x, y in np.argwhere(grid == '#')])

    return S_coords, E_coords, wall_coords, n_rows, n_cols

In [3]:
def get_best_states(S_coords, E_coords, wall_coords):
    queue = deque([(S_coords, 0)])
    best_states = {S_coords: 0}
    min_score = float('inf')

    while queue:
        (x_current, y_current), score = queue.popleft()

        if score > min_score:
            continue

        if (x_current, y_current) == E_coords:
            min_score = min(min_score, score + 1)
            continue

        for dx, dy in directions:
            x_next, y_next = x_current + dx, y_current + dy 

            if (x_next, y_next) in wall_coords:
                continue

            state = (x_next, y_next)
            if state not in best_states or score + 1 < best_states[state]:
                best_states[state] = score + 1
                queue.append((state, score + 1))

    return best_states

In [4]:
def manhattan(x1, y1, x2, y2):
    return abs(x2 - x1) + abs(y2 - y1)

In [9]:
def compute_cheats(best_states, n_steps_max, n_rows, n_cols):
    cheats = {}

    for x0, y0 in tqdm(best_states):
        for x, y in best_states:
            if x - x0 <= n_steps_max and y - y0 <= n_steps_max:
                n_steps_current = best_states[(x, y)] - best_states[(x0, y0)]
                if n_steps_current > 0:
                    n_steps_cheat = manhattan(x0, y0, x, y)
                    if n_steps_cheat <= n_steps_max and n_steps_cheat < n_steps_current:
                        cheats[((x0, y0), (x, y))] = n_steps_current - n_steps_cheat

    return cheats

In [16]:
def main(file, n_steps_max):
    S_coords, E_coords, wall_coords, n_rows, n_cols = parse_input(file)
    best_states = get_best_states(S_coords, E_coords, wall_coords)
    cheats = compute_cheats(best_states, n_steps_max, n_rows, n_cols)

    return len([gain for gain in cheats.values() if gain >= 100])

In [11]:
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

In [13]:
main('input.txt', n_steps_max=2)

100%|██████████| 9465/9465 [00:12<00:00, 761.13it/s] 


1363

In [15]:
main('input.txt', n_steps_max=20)

100%|██████████| 9465/9465 [00:18<00:00, 513.78it/s] 


1007186